# Index
* [Import](#index)
* [Tokenization](#Tokenization)
* [CountVectorizer](#CountVectorizer)
* [Tfidf](#Tfidf)
* [OvsRest](#OvsRest)
    - [LogisticRegression](#LogisticRegression)
    - [SVC](#SVC)
* [Word2Vec](#Word2Vec)
* [BERT](#BERT)
* [USE](#USE)

In [1]:
import time

# import warnings

import numpy as np
import pandas as pd
import gensim

# from gensim import (
#     corpora,
# )  # https://pypi.org/project/gensim/ #https://radimrehurek.com/gensim/auto_examples/core/run_topics_and_transformations.html#sphx-glr-auto-examples-core-run-topics-and-transformations-py
from sklearn import (
    metrics,
    feature_extraction,
    linear_model,
    model_selection,
    preprocessing,
    cluster,
    multiclass,
    svm,
    pipeline,
    exceptions,
    manifold,
)
from sklearn.utils._testing import ignore_warnings
from tensorflow import keras

# import nlpk

In [2]:
class colors:
    PURPLE = "\033[95m"
    BLUE = "\033[94m"
    CYAN = "\033[96m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    RED = "\033[91m"
    ENDC = "\033[0m"
    BOLD = "\033[1m"
    UNDERLINE = "\033[4m"

test https://scikit-learn.org/stable/modules/multiclass.html autre stategie que OneVsRest
https://keras.io/examples/nlp/text_extraction_with_bert/

In [3]:
pip install -q jupyter-black jupyter

Note: you may need to restart the kernel to use updated packages.


In [4]:
# for black formating
import jupyter_black  # pip install jupyter-black jupyter

jupyter_black.load()
# ??jupyter_black.load #show config

In [5]:
LONG = False
df = pd.read_csv(
    "/kaggle/input/p5-nlp/stackoverflowCleaned.csv", index_col=0, low_memory=False
)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118442 entries, 0 to 118441
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Id           118442 non-null  int64 
 1   Title        118442 non-null  object
 2   Body         118442 non-null  object
 3   0            118440 non-null  object
 4   1            118383 non-null  object
 5   2            118292 non-null  object
 6   3            118292 non-null  object
 7   4            76389 non-null   object
 8   5            4 non-null       object
 9   Title_clean  118389 non-null  object
 10  Code         88202 non-null   object
 11  Body_clean   118392 non-null  object
dtypes: int64(1), object(11)
memory usage: 11.7+ MB


# Tokenization

In [6]:
from ipywidgets import interact


@interact(i=(0, df.shape[0] - 1))
def test_tr_print(i):
    print(
        f'Title:{df["Title"][i]}\n{"*"*30}\nTitle_clean:{df["Title_clean"][i]}\n{"*"*30}\nBody:{df["Body"][i]}\n{"*"*30}\nBody_clean:{df["Body_clean"][i]}\n{"*"*30}\nCode:{df["Code"][i]}\n{"*"*30}\nTags:{df.loc[i,"0"]} {df.loc[i,"1"]} {df.loc[i,"2"]} {df.loc[i,"3"]} {df.loc[i,"4"]}'
    )

interactive(children=(IntSlider(value=59220, description='i', max=118441), Output()), _dom_classes=('widget-in…

In [7]:
sum(df["Title_clean"].isna())

53

In [8]:
df[df["Title_clean"].isna()][
    ["Title", "Title_clean", "Body", "Body_clean", "0", "1", "2", "3", "4"]
].head(5)

,Title,Title_clean,Body,Body_clean,0,1,2,3,4
302,Is < faster than <=?,NaN,<p>Is <code>if (a &lt; 901)</code> faster than...,example performance change loop code suppose g...,c++,c,performance,assembly,relational-operators
393,What is setup.py?,NaN,<p>What is <code>setup.py</code> and how can i...,configure,python,pypi,setup.py,python-packaging,NaN
1437,What is related_name used for?,NaN,<p>What is the <code>related_name</code> argum...,argument manytomanyfield foreignkey field exam...,python,django,foreign-keys,many-to-many,NaN
4062,"What is ""android:allowBackup""?",NaN,"<p>Since the <a href=""http://tools.android.com...",adt preview version version lint warning tell ...,android,adt,compiler-warnings,android-lint,android-backup-service
4870,Why is 0 < -0x80000000?,NaN,<p>I have below a simple program:</p>\n\n<pre>...,program condition if(bal int32_min work change...,c,signed,numeric-limits,numeric-conversion,NaN


In [9]:
print(sum(df["Body_clean"].isna()))
df[df["Body_clean"].isna()][
    ["Title", "Title_clean", "Body", "Body_clean", "Code"]
].head(5)

50


,Title,Title_clean,Body,Body_clean,Code
5,What and where are the stack and heap?,stack heap,<ul>\n<li>What are the stack and heap?</li>\n<...,NaN,NaN
168,What is The Rule of Three?,rule,<ul>\n<li>What does <em>copying an object</em>...,NaN,NaN
828,Why are these constructs using pre and post-in...,construct pre behavior,<pre><code>#include &lt;stdio.h&gt;\n\nint mai...,NaN,#include <stdio.h>\n\nint main(void)\n{\n in...
1207,How to use ADB Shell when Multiple Devices are...,use adb shell multiple device connect fail err...,<pre><code>$ adb --help\n</code></pre>\n<hr />...,NaN,$ adb --help\n -s SERIAL use device with give...
1640,What are the advantages of list initialization...,advantage list initialization brace,<pre><code>MyClass a1 {a}; // clearer and ...,NaN,MyClass a1 {a}; // clearer and less error-...


In [10]:
len(df[df["Body_clean"].isna() & df["Title_clean"].isna()])

0

In [11]:
sum((df["Title_clean"] + df["Body_clean"]).isna())

103

In [12]:
def tag_is_in(df: pd.DataFrame(), tag: str, nb_cols: int = 6):
    return eval("|".join(f'(df["{i}"] == "{tag}")' for i in range(0, nb_cols)))


def tags_are_in(df: pd.DataFrame(), tags: list):
    return pd.DataFrame({tag: tag_is_in(df, tag) for tag in tags})


# tags_are_in(df, ["javascript","java"])
tags_are_in(
    df[10001:15050], ["javascript", "java", "c#", "python", "html", "git", "php"]
)

,javascript,java,c#,python,html,git,php
10001,True,False,False,False,True,False,False
10002,False,True,False,False,False,False,False
10003,False,False,True,False,False,False,False
10004,False,False,False,False,True,False,False
10005,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
15045,False,False,False,False,False,False,False
15046,False,False,False,False,False,False,False
15047,True,False,False,False,False,False,False
15048,False,False,False,False,False,False,False


In [13]:
tags = df[["0", "1", "2", "3", "4", "5"]].stack()
target_names = [
    tag
    for tag, freq in tags.reset_index(drop=True).value_counts(normalize=True).items()
    if freq > 0.002
]
if not LONG:
    target_names = target_names[0:20] + ["git"]
target_names

['java',
 'c#',
 'javascript',
 'python',
 'android',
 'c++',
 'ios',
 'html',
 'php',
 '.net',
 'jquery',
 'css',
 'objective-c',
 'c',
 'sql',
 'iphone',
 'asp.net',
 'mysql',
 'linux',
 'node.js',
 'git']

# CountVectorizer

In [14]:
df["Token"] = df["Title_clean"].fillna("") + df["Body_clean"].fillna("")
print(
    f"{len(df[[len(token.split()) > 1 for token in df['Token']]])/len(df):.5%} des questions ont plus d'un tokens"
    if len(df[df["Token"] == ""]) == 0
    else f"{len(df[df['Token'] == ''])} questions n'ont pas de token"
)
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    df["Token"], tags_are_in(df, target_names), random_state=42
)

99.99409% des questions ont plus d'un tokens


In [15]:
for column in y_train.columns:
    t_train = y_train[column].value_counts(normalize=True)
    t_test = y_test[column].value_counts(normalize=True)
    print(f"{colors.BOLD}{column}{colors.ENDC}")
    print(
        f"\t Sur entrainement {colors.GREEN}{t_train[True]:.2%}{colors.ENDC} positif {colors.RED}{t_train[False]:.2%}{colors.ENDC} negatif"
    )
    print(
        f"\t Sur test {colors.GREEN}{t_test[True]:.2%}{colors.ENDC} positif {colors.RED}{t_test[False]:.2%}{colors.ENDC} negatif"
    )
    print("\n")

java
	 Sur entrainement 12.05% positif 87.95% negatif
	 Sur test 12.02% positif 87.98% negatif


c#
	 Sur entrainement 11.80% positif 88.20% negatif
	 Sur test 11.73% positif 88.27% negatif


javascript
	 Sur entrainement 11.77% positif 88.23% negatif
	 Sur test 11.33% positif 88.67% negatif


python
	 Sur entrainement 8.61% positif 91.39% negatif
	 Sur test 8.82% positif 91.18% negatif


android
	 Sur entrainement 7.00% positif 93.00% negatif
	 Sur test 6.91% positif 93.09% negatif


c++
	 Sur entrainement 6.63% positif 93.37% negatif
	 Sur test 6.63% positif 93.37% negatif


ios
	 Sur entrainement 6.25% positif 93.75% negatif
	 Sur test 6.21% positif 93.79% negatif


html
	 Sur entrainement 5.85% positif 94.15% negatif
	 Sur test 5.93% positif 94.07% negatif


php
	 Sur entrainement 5.56% positif 94.44% negatif
	 Sur test 5.49% positif 94.51% negatif


.net
	 Sur entrainement 5.34% positif 94.66% negatif
	 Sur test 5.30% positif 94.70% negatif


jquery
	 Sur entrainement 5.36% positi

In [16]:
best_min_df = 0.00001
best_max_df = 0.995
for min_df in [0.01, 0.05, 0.005, 0.0005, 0.00001] if LONG else [best_min_df]:
    for max_df in [0.95, 0.995, 0.9995] if LONG else [best_max_df]:
        print(f"\n{'*'*30}\n pour min_df:{min_df}; max_df:{max_df}")
        count_vectorizer = feature_extraction.text.CountVectorizer(
            min_df=min_df, max_df=max_df, max_features=None
        )
        #         pipe = pipeline.make_pipeline(feature_extraction.text.CountVectorizer(min_df=min_df, max_df=max_df, max_features=None), feature_extraction.text.TfidfTransformer( norm="l2", use_idf=True, smooth_idf=True, sublinear_tf=False))
        #         feat_train = pipe.transform(X_train["Token"])
        #         print(feat_train.shape)

        token_vectors = count_vectorizer.fit_transform(df["Token"])
        print(
            f"\ttokens unique:\n\t\t{token_vectors.shape[1]} dans title et body regroupé"
        )


******************************
 pour min_df:1e-05; max_df:0.995
	tokens unique:
		53999 dans title et body regroupé


In [17]:
pipeline_tfidf = pipeline.Pipeline(
    [
        (
            "count",
            feature_extraction.text.CountVectorizer(
                min_df=best_min_df, max_df=best_max_df, max_features=None
            ),
        ),
        (
            "tfidf",
            feature_extraction.text.TfidfTransformer(
                norm="l2", use_idf=True, smooth_idf=True, sublinear_tf=False
            ),
        ),
    ]
)

# Tfidf

In [18]:
# tfidf_transformer = feature_extraction.text.TfidfTransformer(
#     norm="l2", use_idf=True, smooth_idf=True, sublinear_tf=False
# ).fit(X_train)
# X_train = tfidf_transformer.transform(X_train)
# print(X_train.shape[0])
# X_test = tfidf_transformer.transform(X_test)
# print(X_test.shape[0])

In [19]:
# y_train["java"].value_counts(normalize=True)[True]

In [20]:
# [text for text in [token for token in X_test[0:5]]]

In [21]:
# clf = linear_model.LogisticRegression(random_state=0, multi_class="ovr").fit(X_train, y_train)

# OvsRest

In [22]:
def save_score(y_true, y_pred, target_names, name) -> pd.DataFrame():
    scores = pd.DataFrame(
        metrics.classification_report(
            y_true, y_pred, target_names=target_names, zero_division=0, output_dict=True
        )
    ).T
    scores.to_csv(f"/kaggle/working/{name}_score.csv")
    return scores

## LogisticRegression

In [23]:
%%time
# with ignore_warnings(category=exceptions.ConvergenceWarning):
#     clf_lr = multiclass.OneVsRestClassifier(
#         linear_model.LogisticRegression(random_state=42)
#     ).fit(X_train, y_train)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.58 µs


In [24]:
pipeline_logisticReg = pipeline.Pipeline(
    [
        ("tfidf", pipeline_tfidf),
        (
            "OvRLogisticReg",
            multiclass.OneVsRestClassifier(
                linear_model.LogisticRegression(random_state=42)
            ),
        ),
    ]
)

In [25]:
%%time
with ignore_warnings(category=exceptions.ConvergenceWarning):
    pipeline_logisticReg.fit(X_train,y_train)

CPU times: user 6min 55s, sys: 6min 55s, total: 13min 51s
Wall time: 3min 42s


In [26]:
pred_lr = pipeline_logisticReg.predict(X_test)

In [27]:
scores_lr = save_score(y_test, pred_lr, target_names, "LogisticRegression")
scores_lr.sort_values(by="support", ascending=False)

,precision,recall,f1-score,support
samples avg,0.406905,0.379385,0.380651,33044.0
micro avg,0.780031,0.455120,0.574841,33044.0
weighted avg,0.762518,0.455120,0.563992,33044.0
macro avg,0.746632,0.425657,0.534277,33044.0
java,0.831630,0.526138,0.644517,3558.0
c#,0.686422,0.366830,0.478138,3473.0
javascript,0.742475,0.529201,0.617954,3356.0
python,0.902315,0.611792,0.729181,2612.0
android,0.933975,0.664059,0.776222,2045.0
c++,0.787489,0.435845,0.561127,1964.0


In [28]:
scores_lr.sort_values(by="precision", ascending=False)

,precision,recall,f1-score,support
android,0.933975,0.664059,0.776222,2045.0
git,0.905473,0.664234,0.766316,274.0
python,0.902315,0.611792,0.729181,2612.0
php,0.877953,0.548248,0.674991,1627.0
java,0.831630,0.526138,0.644517,3558.0
css,0.829815,0.556145,0.665961,1131.0
mysql,0.824356,0.513869,0.633094,685.0
jquery,0.818269,0.553316,0.660202,1538.0
c++,0.787489,0.435845,0.561127,1964.0
ios,0.784535,0.490761,0.603811,1840.0


In [29]:
scores_lr.sort_values(by="recall", ascending=False)

,precision,recall,f1-score,support
git,0.905473,0.664234,0.766316,274.0
android,0.933975,0.664059,0.776222,2045.0
python,0.902315,0.611792,0.729181,2612.0
css,0.829815,0.556145,0.665961,1131.0
jquery,0.818269,0.553316,0.660202,1538.0
php,0.877953,0.548248,0.674991,1627.0
javascript,0.742475,0.529201,0.617954,3356.0
java,0.831630,0.526138,0.644517,3558.0
mysql,0.824356,0.513869,0.633094,685.0
ios,0.784535,0.490761,0.603811,1840.0


In [30]:
X_train

42834     streetview api hiding fullscreen controltry hi...
25716      reason if(1 fooread legacy code see reason write
93960     android studio junit testingupdate accord buil...
106821    string database tableneed generate string digi...
84135     communication loop guimake gui python machine ...
                                ...                        
76820     view inside scroll view increase width pull re...
110268    problem xamarin form getasync(url jsonconvertc...
103694    install mssql driver mac os phpmac run php 5.3...
860       python function definitionnotice look python g...
15795     compare datetime date sql serverdatabase user ...
Name: Token, Length: 88831, dtype: object

In [31]:
y_train[0:5]

,java,c#,javascript,python,android,c++,ios,html,php,.net,...,css,objective-c,c,sql,iphone,asp.net,mysql,linux,node.js,git
42834,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
25716,False,False,False,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
93960,True,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
106821,False,False,True,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,False,True,False
84135,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## SVC

In [32]:
pipeline_scv = pipeline.Pipeline(
    [
        ("tfidf", pipeline_tfidf),
        (
            "OvRSVC",
            multiclass.OneVsRestClassifier(
                svm.SVC(cache_size=1080, max_iter=5_000 if LONG else 1_000, random_state=42)
            ),
        ),
    ]
)

In [33]:
%%time
with ignore_warnings(category=exceptions.ConvergenceWarning):
    pipeline_scv.fit(
        X_train,
        y_train,
    )

CPU times: user 21min 42s, sys: 11.8 s, total: 21min 54s
Wall time: 21min 55s


In [34]:
%%time
pred_scv = pipeline_scv.predict(X_test)
pred_scv

CPU times: user 7min 5s, sys: 1.18 s, total: 7min 6s
Wall time: 7min 4s


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [35]:
scores_scv = save_score(y_test,pred_scv,target_names,"TfidfOvRestSvc")
scores_scv.sort_values(by="precision", ascending=False)

,precision,recall,f1-score,support
git,0.903509,0.751825,0.820717,274.0
c++,0.754601,0.250509,0.376147,1964.0
mysql,0.753704,0.594161,0.664490,685.0
android,0.694884,0.730562,0.712277,2045.0
node.js,0.586552,0.601173,0.593773,682.0
linux,0.544444,0.216176,0.309474,680.0
jquery,0.514717,0.443433,0.476423,1538.0
css,0.457585,0.653404,0.538237,1131.0
python,0.448074,0.703675,0.547513,2612.0
javascript,0.437308,0.338796,0.381800,3356.0


# Word2Vec

In [36]:
df[0:10]["Token"]

0    process array process arraypiece c++ code show...
1    undo commit gitcommit file git push commit ser...
2    delete git branchdelete remote origin bugfix b...
3    difference git pull git fetchdifference git pu...
4    operator c++read hidden features dark corners ...
5                                           stack heap
6    force git pull overwrite fileforce overwrite f...
7    check element hide jquerytoggle visibility ele...
8    use javascript reasoningrun javascript code cr...
9    _ _ _ = _ _include statement try close questio...
Name: Token, dtype: object

In [37]:
# gensim.corpora.Dictionary(token["Token"])

In [38]:
%%time
print("Build & train Word2Vec model ...")
# sentences = [gensim.utils.simple_preprocess(text) for text in df[0:20_000]["Token"]]
sentences = df[0:10_000]["Token"]
w2v_min_count = 1
w2v_size = 300
w2v_window = 5
w2v_epochs = 100
maxlen = 24
w2v_model = gensim.models.Word2Vec(
    min_count=w2v_min_count, window=w2v_window, vector_size=w2v_size, seed=42, workers=1
)
#                                                workers=multiprocessing.cpu_count())
w2v_model.build_vocab(sentences)
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=w2v_epochs)
model_vectors = w2v_model.wv
w2v_words = model_vectors.index_to_key
print("Vocabulary size: %i" % len(w2v_words))
print("Word2Vec trained")

Build & train Word2Vec model ...
Vocabulary size: 133
Word2Vec trained
CPU times: user 1min 56s, sys: 1.67 s, total: 1min 57s
Wall time: 1min 55s


In [39]:
%%time
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(sentences)
x_sentences = keras.preprocessing.sequence.pad_sequences(
    tokenizer.texts_to_sequences(sentences), maxlen=maxlen, padding="post"
)

num_words = len(tokenizer.word_index) + 1
print("Number of unique words: %i" % num_words)

Number of unique words: 23893
CPU times: user 1.42 s, sys: 138 ms, total: 1.56 s
Wall time: 1.84 s


## matrice d'embedding

In [40]:
%%time
print("Create Embedding matrix ...")
w2v_size = 300
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
embedding_matrix = np.zeros((vocab_size, w2v_size))
i = 0
j = 0

for word, idx in word_index.items():
    i += 1
    if word in w2v_words:
        j += 1
        embedding_vector = model_vectors[word]
        if embedding_vector is not None:
            embedding_matrix[idx] = model_vectors[word]

word_rate = np.round(j / i, 4)
print("Word embedding rate : ", word_rate)
print("Embedding matrix: %s" % str(embedding_matrix.shape))

Create Embedding matrix ...
Word embedding rate :  0.0034
Embedding matrix: (23893, 300)
CPU times: user 65.9 ms, sys: 0 ns, total: 65.9 ms
Wall time: 65.5 ms


## Creation du modèle

In [41]:
%%time
input = keras.layers.Input(shape=(len(x_sentences), maxlen), dtype="float64")
word_input = keras.layers.Input(shape=(maxlen,), dtype="float64")
word_embedding = keras.layers.Embedding(
    input_dim=vocab_size,
    output_dim=w2v_size,
    weights=[embedding_matrix],
    input_length=maxlen,
)(word_input)
word_vec = keras.layers.GlobalAveragePooling1D()(word_embedding)
embed_model = keras.models.Model([word_input], word_vec)

embed_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 24)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 24, 300)           7167900   
_________________________________________________________________
global_average_pooling1d (Gl (None, 300)               0         
Total params: 7,167,900
Trainable params: 7,167,900
Non-trainable params: 0
_________________________________________________________________
CPU times: user 154 ms, sys: 90.2 ms, total: 244 ms
Wall time: 175 ms


2022-12-18 12:26:39.674981: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [42]:
embeddings = embed_model.predict(x_sentences)
embeddings.shape

2022-12-18 12:26:39.939419: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


(10000, 300)

In [43]:
%%time
with ignore_warnings(category=exceptions.ConvergenceWarning):
    OvsRSVC_word2vec = multiclass.OneVsRestClassifier(
        svm.SVC(cache_size=1080, max_iter=5_000 if LONG else 1_000, random_state=42)
    ).fit(embeddings, tags_are_in(df[0:10_000], target_names))

CPU times: user 1min 6s, sys: 233 ms, total: 1min 6s
Wall time: 1min 6s


In [44]:
%%time
pred_word2vec = OvsRSVC_word2vec.predict(embeddings)
pred_word2vec

CPU times: user 1min 26s, sys: 10.3 ms, total: 1min 26s
Wall time: 1min 26s


array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       ...,
       [1, 0, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0]])

In [45]:
scores_word2vec = save_score(
    tags_are_in(df[0:10_000], target_names), pred_word2vec, target_names, "word2vec"
)
scores_word2vec.sort_values(by="precision", ascending=False)

,precision,recall,f1-score,support
jquery,1.000000,0.010076,0.019950,397.0
.net,1.000000,0.003781,0.007533,529.0
linux,1.000000,0.007018,0.013937,285.0
mysql,1.000000,0.011236,0.022222,178.0
sql,1.000000,0.002762,0.005510,362.0
c,1.000000,0.022422,0.043860,223.0
css,1.000000,0.002710,0.005405,369.0
git,1.000000,0.007067,0.014035,283.0
node.js,1.000000,0.044118,0.084507,272.0
php,1.000000,0.006849,0.013605,292.0


In [46]:
def ARI_fct(features, target_names, y):
    time1 = time.time()
    num_labels = len(target_names)
    tsne = manifold.TSNE(
        n_components=2,
        perplexity=30,
        n_iter=2000,
        init="random",
        learning_rate=200,
        random_state=42,
    )
    X_tsne = tsne.fit_transform(features)

    # Détermination des clusters à partir des données après Tsne
    cls = cluster.KMeans(n_clusters=num_labels, n_init=100, random_state=42)
    cls.fit(X_tsne)
    #     ARI = np.round(metrics.adjusted_rand_score(y, cls.labels_), 4)
    time2 = np.round(time.time() - time1, 0)
    scores = pd.DataFrame(
        metrics.classification_report(
            np.argmax(y, axis=1),
            np.argmax(cls.labels_, axis=1),
            target_names=target_names,
            zero_division=0,
            output_dict=True,
        )
    )
    print(scores)
    print("time : {:.2f}".format(ARI, time.time() - time1))

    return scores, X_tsne, cls.labels_


# visualisation du Tsne selon les vraies catégories et selon les clusters
def TSNE_visu_fct(X_tsne, y, labels, ARI):
    fig = plt.figure(figsize=(15, 6))

    ax = fig.add_subplot(121)
    scatter = ax.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y, cmap="Set1")
    ax.legend(
        handles=scatter.legend_elements()[0],
        labels=l_cat,
        loc="best",
        title="Categorie",
    )
    plt.title("Représentation des tweets par catégories réelles")

    ax = fig.add_subplot(122)
    scatter = ax.scatter(X_tsne[:, 0], X_tsne[:, 1], c=labels, cmap="Set1")
    ax.legend(
        handles=scatter.legend_elements()[0],
        labels=set(labels),
        loc="best",
        title="Clusters",
    )
    plt.title("Représentation des tweets par clusters")

    plt.show()
    print("ARI : ", ARI)

In [47]:
%%time
# ARI, X_tsne, labels = ARI_fct(
#     embeddings, target_names, y=tags_are_in(df[0:20000], target_names)
# )

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 8.58 µs


In [48]:
# %%time
# tsne = manifold.TSNE(
#     n_components=2,
#     perplexity=30,
#     n_iter=2000,
#     init="random",
#     learning_rate=200,
#     random_state=42,
# )
# X_tsne = tsne.fit_transform(embeddings)

In [49]:
# X_tsne

In [50]:
# %%time
# # Détermination des clusters à partir des données après Tsne
# time_start = time.time()
# cls = cluster.KMeans(n_clusters=len(target_names), n_init=100, random_state=42)
# cls.fit(X_tsne)
# #     ARI = np.round(metrics.adjusted_rand_score(y, cls.labels_), 4)
# scores = pd.DataFrame(
#     metrics.classification_report(
#         tags_are_in(df[0:20000], target_names),
#         np.argmax(cls.labels_, axis=1),
#         target_names=target_names,
#         zero_division=0,
#         output_dict=True,
#     )
# )
# print(scores)
# print("time : {:.2f}".format(time.time() - time_start))

In [51]:
# cls.labels_

In [52]:
# len(cls.labels_)

In [53]:
# scores = pd.DataFrame(
#     metrics.classification_report(
#         tags_are_in(df[0:20000], target_names),
#         np.argmax(cls.labels_, axis=1),
#         target_names=target_names,
#         zero_division=0,
#         output_dict=True,
#     )
# )

# BERT

# USE